# Data Gathering

In [ ]:
%matplotlib widget

In [ ]:
from utils.io import gen_id_dated, gdf_viewer, gdf_geom, gdf_merger, gdf_filter, na_col_drop, na_line_drop
import re, os
import numpy as np
import geopandas as gpd
import pandas as pd
#from shapely.geometry import Point
import datetime as dtm
import matplotlib.pyplot as plt
from definitions import ROOT_DIR

In [ ]:
def files_search(work_dir, files_dict, prefix='', skip=None, details=False):   
    
    if skip is None:
        skip = "we don't want to skip a word"
        
    for k in files_dict.keys():
        tmp_list = []
        for p,d,f in os.walk(work_dir):
            for x in f:
                add = False
                if re.search(prefix,x,re.I) and not re.search(skip,x,re.I):
                    add = True
                    i = str(f'{p}/{x}')
                else:
                    add = False
                    i=''
                    
                if re.search(k,i,re.I) and add:
                    tmp_list.append(i)
        tmp_list.sort()
        files_dict.update({k:tmp_list})

    for k,v in files_dict.items():
        print(k,' \t: ',len(v))
    
    if details: # Look filenames
        which = files_dict.keys()

        for w in which:
            print('\n+++++++++++++++++')
            print(f'+  {w.upper()}\t+ ')
            print('+++++++++++++++++')
            [print(i, '-', x) for i, x in enumerate(files_dict[w], 0)]    


In [ ]:
def check_col(data):
    cols_idx = []
    
    class DoubledColumns(Exception):
        """Merging process doubled column(s) still remain. Check and drop them before continue"""
        pass
    
    for i in range(len(data.columns)):
        if re.search('_x|_y', list(data.columns)[i]):
            cols_idx.append(i)
    
    if len(cols_idx) != 0 :
        raise DoubledColumns(f'Merging process doubled column(s) still remain.'
                             f'\nCheck and drop them before continue ! Doubled columns position {cols_idx}')

In [ ]:
def distinct_obj_test(df1, df2, on='ID', how='outer', dist_max=1):
    test_distinct = df1.merge(df2, on=on, how=how)

    dist_max = dist_max
    
    if 'X' in list(df1.columns) and 'X' in list(df2.columns):
        for idx in test_distinct.index:
            distinct_objects = True
            if not pd.isnull(test_distinct.loc[idx,'X_x']) and not pd.isnull(test_distinct.loc[idx,'X_y']):
                dist = (test_distinct.loc[idx,'X_x'] - test_distinct.loc[idx,'X_y']) ** 2 + (test_distinct.loc[idx,'Y_x'] - test_distinct.loc[idx,'Y_y']) ** 2
                if dist <= (dist_max) ** 2:  # consider as same object
                    distinct_objects = False
            else:
                distinct_objects = False
            test_distinct.loc[idx, 'Distinct_obj'] = distinct_objects

        test_distinct.insert(1,'Distinct_obj', test_distinct.pop('Distinct_obj') )

        gdf_viewer(test_distinct)
    else:
        print('Cannot proceed ! No position data in one of the dataframe')

In [ ]:
def create_df(file1, file2): # find another name for this function
    """
    create dataframes from files and test if they contain position informations
    """
    
    df1 = pd.read_csv(file1, delimiter=',')
    df2 = pd.read_csv(file2, delimiter=',')
    
    print(f"df1 : {file1.replace(work_dir,'')} \ndf2 : {file2.replace(work_dir,'')}\n")

    if 'X' in list(df1.columns): print('df1 - Position data')
    else: print('df1 - No position data')
    if 'X' in list(df2.columns): print('df2 - Position data')
    else: print('df2 - No position data')
    
    return df1, df2

In [ ]:
def validate_data(data, data_to_check, valid_data, col, idx_list, valid_col):
    
    old_idx_col = 'Source_index'
    
    for col, idx_list in valid_data.items():
        if old_idx_col in data_to_check.columns:
            idx = data_to_check.loc[i, old_idx_col]
            data.loc[idx, col] = data_to_check.loc[i, valid_col]
        else:
            raise NameError(f"Dataframe to check must contain a column named : '{old_idx_col}'!")

    data_to_check.drop(index=idx_list, inplace=True)
    data_to_check.reset_index(drop=True, inplace=True)
    print(f"Operation done ")
    
    #return data

In [ ]:
def double_objects_check(data):
    uniq_ID = []
    dbl_ID = []
    idx_ = []
    qdf = pd.DataFrame()

    for i in data.index:
        id_ = data.loc[i, 'ID']

        if id_ not in uniq_ID:
            uniq_ID.append(id_)
        elif id_ not in dbl_ID:
            dbl_ID.append(id_)
        else:
            idx_.append(i)

    for i in dbl_ID:
        qdf = qdf.append(data.query(f"ID=='{i}'"))

    return qdf

## Files reading

In [ ]:
work_dir = ROOT_DIR+'/CF_data/Result_traitem/'
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [ ]:
# create my dictionary structure to retrieve good files (Keynames !!!)
files_dict={'Borehole':0,'Piezometer':0,'Piezair':0,'Trench':0,'Litho':0,'Equipm':0,
        'Measure':0,'Sample':0,'Analysis':0,'facility':0}

In [ ]:
files_search(work_dir, files_dict, prefix='', skip='source')

In [ ]:
how=['inner', 'outer', 'left', 'right']

In [ ]:
f = False
t = True

# Boreholes

Some corrections todo in 'data organization':
- correct extraction in the file 2 -> Samples
- file 4 and file 5 are the same in result (check it)
- try to concatenate file 1 with piezo (if possible because no position)
- check processing for 'refus and 'type_refus' (every object)

In [ ]:
# keys: Borehole','Piezometer','Litho', 'Trench','Equipm','Measure','Sample','Analysis','facility'
files_dict['Borehole']

In [ ]:
key='Borehole'
save_file = f'Merged_Boreholes.csv'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus', 'Societe'] #columns of interest
boreholes = pd.DataFrame() # for saving object info after last merging
print(len(files_dict[key]), 'files')

In [ ]:
file1= files_dict[key][1]
file2= files_dict[key][3]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df2.ID=df2.ID.apply(lambda x: 'F'+x) # name recent (2019) boreholes

In [ ]:
distinct_obj_test(df1, df2, dist_max=1)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)
check_col(mdf) # check if columns with '..._x' or '..._y' are still present and raise an error

In [ ]:
err_df

In [ ]:
gdf_viewer(mdf, rows=10, un_val='ID', view=t)

#### boreholes merging

In [ ]:
boreholes = mdf.copy() #saving

In [ ]:
file1= files_dict[key][2]
file2= files_dict[key][4]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
distinct_obj_test(df1, df2, dist_max=1)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID')
check_col(mdf)

In [ ]:
mdf['ID_date'] = mdf['ID_date'].apply(lambda x: str(x).upper())

In [ ]:
# mdf['Long_for'] = mdf[['Profondeur', 'Long_for']].apply(lambda x: x[0] if pd.isnull(x[1]) else x[1], axis=1)
# mdf.drop(columns=['Profondeur'], inplace=True)

In [ ]:
gdf_viewer(mdf, rows=3, cols=15, un_val='ID', view=t)

#### boreholes merging

In [ ]:
boreholes.drop('index', axis='columns', inplace=True)

In [ ]:
boreholes.drop('split_distinct', axis='columns', inplace=True)

In [ ]:
boreholes, err_df=gdf_merger(boreholes, mdf, how=how[1], on='ID', dist_max=2)
check_col(boreholes)

In [ ]:
boreholes.loc[[12,13,14,15,16],:]

In [ ]:
err_df

In [ ]:
# TODO : Continue to write validate data function

In [ ]:
validate_data(boreholes, err_df, valid_data={'Date_ouv_x': [0]}, col='Long_for', idx_list=[0,1,2], valid_col='Long_for_y')

In [ ]:
validate_data?

In [ ]:
err_df

In [ ]:
gdf_viewer(boreholes, rows=3, cols=15, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][5]
file2= files_dict[key][0]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
distinct_obj_test(df1, df2, dist_max=1)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', )
check_col(mdf) # check if columns with '..._x' or '..._y' are still present and raise an error

In [ ]:
mdf['Long_for'] = mdf[['Profondeur', 'Long_for']].apply(lambda x: x[0] if pd.isnull(x[1]) else x[1], axis=1)
mdf.drop(columns=['Profondeur'], inplace=True)

In [ ]:
gdf_viewer(mdf, rows=10, un_val='ID', view=t)

#### Last boreholes merging

In [ ]:
boreholes, err_df=gdf_merger(boreholes, mdf, how=how[1], on='ID')
check_col(boreholes)

In [ ]:
err_df # i think there are not the same, but no date or postition to distinguish them !
# --> check boreholes sheets (pdf)

In [ ]:
gdf_viewer(boreholes, rows=3, un_val='ID', view=t)

####  $\color{red}{\textbf{Save final Boreholes data}}$

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
boreholes.to_csv(save_dir+save_file, index=False)

# Piezometers

Some corrections todo in 'data organization':

In [ ]:
key='Piezometer'
save_file = f'Merged_Piezometers.csv'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus'] #columns of interest
piezometers = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
file1= files_dict[key][0]
file2= files_dict[key][1]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)
check_col(mdf)

##### check and validate duplicate objects
- The function "gdf_filter()" doesn't work in some cases, so we use function "doubled_objects_check()"
- we have same objects Names but differents by positions here

In [ ]:
mdf, check = gdf_filter(mdf, position=True, id_on='ID', expression='sup|prof', dist_crit=1, drop=True, rapp_val=1)

In [ ]:
double_objects_check(mdf)

In [ ]:
drop_id = [2,25,30] # objects are seemingly the same, but is it possible to get 2 objects so close (~ 1m)?
mdf.drop(index=drop_id, inplace=True)
mdf.reset_index(drop=True, inplace=True)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

##### Piezometers merging 

In [ ]:
piezometers = mdf.copy() #saving

In [ ]:
file1= files_dict[key][2]
file2= files_dict[key][3]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
double_objects_check(piezometers)

In [ ]:
drop_id = [292, 293]
piezometers.drop(index=drop_id, inplace=True)
gdf_viewer(piezometers, rows=5, un_val='ID', view=f)

In [ ]:
file1= files_dict[key][4]
file2= files_dict[key][5]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
piezometers, check = gdf_filter(piezometers, position=True, id_on='ID', expression='sup|prof', dist_crit=1, drop=True)
#gdf_viewer(piezometers, rows=5, un_val='ID', view=t)

In [ ]:
double_objects_check(piezometers)

In [ ]:
drop_id = [2,4,30,94,106]
piezometers.drop(index=drop_id, inplace=True)
gdf_viewer(piezometers, rows=5, un_val='ID', view=f)

In [ ]:
file1= files_dict[key][6]
file2= files_dict[key][9]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df2['ID'] = df2.ID.astype('object')

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][10]
file2= files_dict[key][11]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1['ID'] = df1.ID.astype('object')

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
err_df

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][12]
file2= files_dict[key][13]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
err_df

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][14]
file2= files_dict[key][15]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
piezometers, err_df=gdf_merger(piezometers, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
err_df

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][16]
df1 = pd.read_csv(file1, delimiter=',')

print(f"df1 : {file1.replace(work_dir,'')}")
gdf_viewer(df1, rows=3, un_val='ID', view=t)

#### Last merging

In [ ]:
piezometers, err_df=gdf_merger(piezometers, df1, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
err_df

In [ ]:
gdf_viewer(piezometers, rows=3, cols=13, un_val='ID', view=t)

####  $\color{red}{\textbf{Save final Piezometers data}}$

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
piezometers.to_csv(save_dir+save_file, index=False)

==========================================================================================================

# Unknown facilities

In [ ]:
key='facility'
save_file = f'Merged_Facilites_unknw.csv'
#coi=['ID','X','Y','Z','Litho_top','Litho_base','Description']  #columns of interest
facilities = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
file1= files_dict[key][0]
file2= files_dict[key][3]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID')#, step_merge
check_col(mdf)

In [ ]:
facilities = mdf.copy() #saving

In [ ]:
file1= files_dict[key][1]
df1 = pd.read_csv(file1, delimiter=',')

print(f"df1 : {file1.replace(work_dir,'')}")
gdf_viewer(df1, rows=3, un_val='ID', view=t)

In [ ]:
facilities, err_df=gdf_merger(facilities, df1, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(facilities, rows=3, un_val='ID', view=t)

####  $\color{red}{\textbf{Save final Unknown Facilities data}}$

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
facilities.to_csv(save_dir+save_file, index=False)

# Lithologies

Do not add parameter 'dist_max' when merging without considering position !!! otherwise, unuseless rows added

In [ ]:
key='Litho'
save_file = f'Merged_Lithologies.csv'
coi=['ID','X','Y','Z','Litho_top','Litho_base','Description']  #columns of interest
lithologies = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
file1= files_dict[key][0]
file2= files_dict[key][3]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID')#, step_merge
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
gdf_viewer(err_df, rows=5, un_val='ID', view=t) #err_df.ID.unique()

In [ ]:
common_cols = list(set(df1.columns) & set(df2.columns))
test1 = df1.merge(df2, how = 'inner', on='ID')
test2 = df1.merge(df2, how = 'outer', on='ID', indicator=True).loc[lambda x : x.query('_merge =="right_only" or _merge=="left_only"').index]
test3 = test1.merge(test2, how = 'outer', on='ID')
test4 = df1.merge(df2, how = 'outer', on=list(common_cols))
print((len(test1), len(test2), len(test3)))
gdf_viewer(test4)

In [ ]:
lithologies = mdf.copy() #saving

In [ ]:
file1= files_dict[key][2]
file2= files_dict[key][4]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID')
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

##### Lithologies merging 

In [ ]:
lithologies, err_df=gdf_merger(lithologies, mdf, how=how[1], on='ID')
check_col(mdf)

In [ ]:
gdf_viewer(lithologies, rows=10, cols=15, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][5]
file2= files_dict[key][6]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
gdf_viewer(df1.merge(df2, how='inner', on='ID'), rows=5, cols=15, un_val='ID', view=t)

In [ ]:
gdf_viewer(df1.merge(df2, how = 'outer', on='ID',indicator=True), rows=5, cols=15, un_val='ID', view=t)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(err_df, rows=5, un_val='ID', view=t) #err_df.ID.unique()

##### Lithologies merging 

In [ ]:
lithologies, err_df=gdf_merger(lithologies, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(lithologies, rows=3, un_val='ID', view=t)

In [ ]:
stop

# Equipments

We must also retrieve equipments information from boreholes and piezometers

In [ ]:
key='Equipm'
save_file = f'Merged_Equipments.csv'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus'] #columns of interest
equipments = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
file1= files_dict[key][0]
file2= files_dict[key][1]

df1, df2 = create_df(file1, file2)
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, err_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)
check_col(mdf)

##### check and validate duplicate objects
- The function "gdf_filter()" doesn't work in some cases, so we use function "doubled_objects_check()"
- we have same objects Names but differents by positions here

In [ ]:
mdf, check = gdf_filter(mdf, position=True, id_on='ID', expression='sup|prof', dist_crit=1, drop=True, rapp_val=1)

In [ ]:
double_objects_check(mdf)

In [ ]:
drop_id = [2,25,30] # objects are seemingly the same, but is it possible to get 2 objects so close (~ 1m)?
mdf.drop(index=drop_id, inplace=True)
mdf.reset_index(drop=True, inplace=True)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

##### Piezometers merging 

In [ ]:
piezometers = mdf.copy() #saving

# Samples

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)

# Measures

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)

# Analysis

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)